In [1]:
!python -V

Python 3.13.3


In [11]:
import pandas as pd

In [12]:
import pickle

In [13]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
## Question 1
df.shape[0]

3403766

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
## Question 2
df_train.shape[0]

3316216

In [11]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

df_train[categorical] = df_train[categorical].astype('str')

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

#val_dicts = df_val[categorical + numerical].to_dict(orient='records')
#X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
#y_val = df_val[target].values

In [13]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## Question 5. Train a model

We will now train a linear regression model using the same code as in homework 1.

- Fit a dict vectorizer.
- Train a linear regression with default parameters.
- Use pick up and drop off locations separately, don't create a combination feature.
Let's now use it in the pipeline. We will need to create another transformation block, and return both the dict vectorizer and the model.

What's the intercept of the model?

Hint: print the ``intercept_`` field in the code block

In [14]:
lr.intercept_

np.float64(23.847909256918335)

In [14]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment-hw")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1749311819365, experiment_id='2', last_update_time=1749311819365, lifecycle_stage='active', name='nyc-taxi-experiment-hw', tags={}>

In [ ]:
import mlflow
from sklearn.linear_model import LinearRegression
with mlflow.start_run():
    df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    
    dv = DictVectorizer()
    
    df_train[categorical] = df_train[categorical].astype('str')
    
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)
    
    target = 'duration'
    y_train = df_train[target].values
    
    mlflow.set_tag("hw3", "pastor")

    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)
    rmse = root_mean_squared_error(y_pred, y_train)
    mlflow.log_metric("rmse", rmse)

2025/06/07 15:58:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/07 15:58:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [1]:
import xgboost as xgb

In [6]:
from pathlib import Path
import mlflow

In [4]:
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [7]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

MlflowException: Could not find experiment with ID 0